<a href="https://colab.research.google.com/github/ShreyasheeSinha/Determining-Robustness-of-NLU-Models/blob/main/analysis_notebooks/Gold_label_change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook used to generate the excel sheet to carry out gold-label reannotation. 

https://docs.google.com/spreadsheets/d/1-525HwmIqhiWw1x4SgKaZvqizJMUEU4Y/edit?usp=share_link&ouid=115562379648834964427&rtpof=true&sd=true

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import json
import numpy as np

In [ ]:
def read_jsonl_file(path):
  with open(path, 'r') as json_file:
      json_list = list(json_file)

  results = []
  for json_str in json_list:
      results.append(json.loads(json_str))
  return results

In [ ]:
test_data = pd.DataFrame(read_jsonl_file("/content/gdrive/MyDrive/paraphraser_work/MTurk data/Generated_dataset/rte_test_paraphrased_p_d_or_h_d_sorted.jsonl"))
test_data = test_data.fillna(value=np.nan)

In [ ]:
df = test_data

In [ ]:
ph = df.copy()
ph_ = df.copy()
p_h = df.copy()
p_h_ = df.copy()

ph = ph.loc[ph['s1_para_id'].isnull() & ph['s2_para_id'].isnull()]
ph_ = ph_.loc[ph_['s1_para_id'].isnull() & ~ph_['s2_para_id'].isnull()]
p_h = p_h.loc[~p_h['s1_para_id'].isnull() & p_h['s2_para_id'].isnull()]
p_h_ = p_h_.loc[~p_h_['s1_para_id'].isnull() & ~p_h_['s2_para_id'].isnull()]

In [ ]:
print(len(ph), len(ph_), len(p_h), len(p_h_))

914 631 415 132


In [ ]:
test_data

,dataset,corpus_sent_id,gold_label,task,sentence1,sentence2,s2_para_id,silver_label,s1_para_id
0,RTE2_test.column,485,True,SUM,Insurgent attacks on pipelines have brought oi...,Insurgents have launched attacks on Iraq 's oi...,NaN,NaN,NaN
1,RTE2_test.column,485,True,SUM,Insurgent attacks on pipelines have brought oi...,Insurgents have launched attacks on Iraq's oil...,RTE2_test.column_s2_485_para1,True,NaN
2,RTE3_test.column,681,True,SUM,They refused to appear in the World Court 10 y...,The court heard US appeals for the release of ...,NaN,NaN,NaN
3,RTE3_test.column,681,True,SUM,They refused to appear in the World Court 10 y...,The court heard US appeals to release of hosta...,RTE3_test.column_s2_681_para1,True,NaN
4,RTE3_test.column,291,False,IR,The United States is engaged in extensive inte...,United states is engaged in illegal technology...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2087,RTE3_test.column,543,True,QA,"Gouverneur George W. Bush's wife, Laura, and h...",Laura is George W. Bush's wife.,RTE3_test.column_s2_543_para1,True,RTE3_test.column_s1_543_para1
2088,RTE3_test.column,661,False,SUM,"Leakey believed Kenya 's wildlife , which unde...",Kenya 's wildlife is managed in a profitable m...,NaN,NaN,NaN
2089,RTE3_test.column,661,False,SUM,"Leakey believed that Kenya's wildlife, which u...",Kenya 's wildlife is managed in a profitable m...,NaN,False,RTE3_test.column_s1_661_para1
2090,RTE3_test.column,214,False,IR,Almost a century of French history has come to...,French conscientious objectors are given speci...,NaN,NaN,NaN


In [ ]:
grouped_by_data_sent_corpus = test_data.groupby(['dataset', 'corpus_sent_id'])

In [ ]:
analysis_dataframe = pd.DataFrame(columns=["dataset_corpus_id", "Original Pair (P and H)", "Gold label", "Paraphrased pair", "New gold label", "s1_para_id", "s2_para_id"])

In [ ]:
for name, group in grouped_by_data_sent_corpus:
  p_h_pair = group[(group['s1_para_id'].isnull()) & (group['s2_para_id'].isnull())]
  # print(p_h_pair.iloc[0]["dataset"], p_h_pair.iloc[0]["corpus_sent_id"])
  paraphrased_pairs = group[~((group['s1_para_id'].isnull()) & (group['s2_para_id'].isnull()))]

  for index, row in paraphrased_pairs.iterrows():
    dataframe_row = {}
    original_dataset_corpus_id = p_h_pair.iloc[0]["dataset"] + "_" + p_h_pair.iloc[0]["corpus_sent_id"].astype(str)
    original_premise = p_h_pair.iloc[0]["sentence1"]
    original_hypothesis = p_h_pair.iloc[0]["sentence2"]
    original_gold_label = p_h_pair.iloc[0]["gold_label"]
    paraphrased_premise_id = row["s1_para_id"]
    paraphrased_premise = row["sentence1"]
    paraphrased_hypothesis_id = row["s2_para_id"]
    paraphrased_hypothesis = row["sentence2"]
    paraphrased_gold_label = row["gold_label"]
    original_pair = "P: " + original_premise + "\nH: " + original_hypothesis
    paraphrased_pair = "P: " + paraphrased_premise + "\nH: " + paraphrased_hypothesis
    dataframe_row["dataset_corpus_id"] = original_dataset_corpus_id
    dataframe_row["Original Pair (P and H)"] = original_pair
    dataframe_row["Gold label"] = original_gold_label
    dataframe_row["Paraphrased pair"] = paraphrased_pair
    dataframe_row["New gold label"] = paraphrased_gold_label
    dataframe_row["s1_para_id"] = paraphrased_premise_id
    dataframe_row["s2_para_id"] = paraphrased_hypothesis_id
    analysis_dataframe = analysis_dataframe.append(dataframe_row, ignore_index=True)
  # print(analysis_dataframe)
  # break


In [ ]:
analysis_dataframe

,dataset_corpus_id,Original Pair (P and H),Gold label,Paraphrased pair,New gold label,s1_para_id,s2_para_id
0,RTE1_test.column_25,"P: The film was the evening 's big winner , ba...",False,"P: The film was the big winner of the evening,...",False,RTE1_test.column_s1_25_para1,NaN
1,RTE1_test.column_49,P: Kidnappings in Argentina have increased mor...,True,P: Kidnappings in Argentina have increased mor...,True,NaN,RTE1_test.column_s2_49_para1
2,RTE1_test.column_75,"P: At the close , the Dow Jones industrial ave...",True,"P: At the close , the Dow Jones industrial ave...",True,NaN,RTE1_test.column_s2_75_para1
3,RTE1_test.column_82,P: The Los Angeles Lakers won their 3 rd strai...,True,P: The Los Angeles Lakers won their 3 rd strai...,True,NaN,RTE1_test.column_s2_82_para1
4,RTE1_test.column_82,P: The Los Angeles Lakers won their 3 rd strai...,True,P: The Los Angeles Lakers won their 3rd straig...,True,RTE1_test.column_s1_82_para1,NaN
...,...,...,...,...,...,...,...
1173,RTE3_test.column_787,"P: In Moscow , Andrei Kokoshin , the former Ru...",True,"P: In Moscow , Andrei Kokoshin , the former Ru...",True,NaN,RTE3_test.column_s2_787_para1
1174,RTE3_test.column_788,P: The U.S. will seek to field a limited numbe...,False,P: The U.S. will seek to field a limited numbe...,False,NaN,RTE3_test.column_s2_788_para1
1175,RTE3_test.column_796,P: It has pledged to allow the yuan to float m...,True,P: It has pledged to allow the yuan to float m...,True,NaN,RTE3_test.column_s2_796_para1
1176,RTE3_test.column_798,"P: The world 's largest consumer of oil , US i...",False,"P: The world's largest consumer of oil, US imp...",False,RTE3_test.column_s1_798_para1,NaN


In [ ]:
# analysis_dataframe.to_excel("/content/gdrive/MyDrive/paraphraser_work/Gold_label_change/gold_label_change_analysis.xlsx", sheet_name='Sheet_name_1', index=False)